In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [18]:
# load dataset and parse into inputs and outputs
data = pd.read_csv('monte_carlo_results.csv')
X = data[['tau_tot','omega']].values
Y = data[['escape_fraction','mean_scatterings']].values

In [19]:
# create training and testing datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,
random_state=42)

In [20]:
# normalize the data, accounting for range variation in inputs
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

In [21]:
# build model with 3 linear
class RegressionNet(nn.Module): 
    # initialize layers to be used
    def __init__(self):
        super(RegressionNet, self).__init__()
        self.fc1 = nn.Linear(2, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2)
        self.relu = nn.ReLU()
    
    # arrange layers
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [22]:
# set model, error function, and learning rate
model = RegressionNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [23]:
# numpy to pytorch tensor
X_train_tensor = torch.FloatTensor(X_train_s)
Y_train_tensor = torch.FloatTensor(Y_train)
X_test_tensor = torch.FloatTensor(X_test_s)
Y_test_tensor = torch.FloatTensor(Y_test)

In [24]:
# split training dataset, 10% used for validation
val_size = int(0.1 * len(X_train_tensor))
train_size = len(X_train_tensor) - val_size
train_dataset = TensorDataset(X_train_tensor[:train_size], Y_train_tensor[:train_size])
val_dataset = TensorDataset(X_train_tensor[train_size:], Y_train_tensor[train_size:])

# load validation and training sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [25]:
# training loop

epochs = 200
for epoch in range(epochs):
    model.train()

    # train model
    for X_batch, Y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    val_loss = 0
    val_mae = 0
    with torch.no_grad():
        for X_val, Y_val in val_loader:
            outputs = model(X_val)
            val_loss += criterion(outputs, Y_val).item()
            val_mae += torch.mean(torch.abs(outputs - Y_val)).item()
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Val Loss: {val_loss/len(val_loader):.4f}, Val MAE: {val_mae/len(val_loader):.4f}')


Epoch 20/200, Val Loss: 0.0183, Val MAE: 0.0686
Epoch 40/200, Val Loss: 0.0181, Val MAE: 0.0454
Epoch 60/200, Val Loss: 0.0158, Val MAE: 0.0593
Epoch 80/200, Val Loss: 0.0053, Val MAE: 0.0524
Epoch 100/200, Val Loss: 0.0015, Val MAE: 0.0232
Epoch 120/200, Val Loss: 0.0024, Val MAE: 0.0309
Epoch 140/200, Val Loss: 0.0045, Val MAE: 0.0404
Epoch 160/200, Val Loss: 0.0024, Val MAE: 0.0368
Epoch 180/200, Val Loss: 0.0027, Val MAE: 0.0360
Epoch 200/200, Val Loss: 0.0011, Val MAE: 0.0224


In [26]:
model.eval()
with torch.no_grad():
    Y_pred = model(X_test_tensor).numpy()
    test_loss = criterion(model(X_test_tensor), Y_test_tensor).item()
    test_mae = torch.mean(torch.abs(model(X_test_tensor) - Y_test_tensor)).item()

print('Test MAE:', test_mae)

# Compute percent errors
percent_errors = 100.0 * np.abs((Y_test - Y_pred) / (Y_test + 1e-12))
print('Median percent error (escape_fraction):', np.median(percent_errors[:,0]))
print('Median percent error (mean_scatterings):', np.median(percent_errors[:,1]))

Test MAE: 0.029656942933797836
Median percent error (escape_fraction): 5.6798435883382865
Median percent error (mean_scatterings): 5.072514143589816
